```
pip3 install .
```

from within the Autorecommender folder.

Requirements:

-        "Keras >= 2.1.3",
-        "pandas >= 0.23.0",
-        "numpy >= 1.14.2",
-        "tensorflow >= 1.4.0",
-        "scipy >= 1.1.0"

In [1]:
import os
print(os.getcwd())

/home/jovyan/work/POC/models/notebooks


In [2]:
import pandas as pd
import numpy as np
import math

In [3]:
cd /home/jovyan/work/POC/models/recommender/AutoRecommender/autorecommender/models

/home/jovyan/work/POC/models/recommender/AutoRecommender/autorecommender/models


In [4]:
import sys
!{sys.executable} -m pip install -I pandas==1.1.0

  Could not find a version that satisfies the requirement pandas==1.1.0 (from versions: 0.1, 0.2b0, 0.2b1, 0.2, 0.3.0b0, 0.3.0b2, 0.3.0, 0.4.0, 0.4.1, 0.4.2, 0.4.3, 0.5.0, 0.6.0, 0.6.1, 0.7.0rc1, 0.7.0, 0.7.1, 0.7.2, 0.7.3, 0.8.0rc1, 0.8.0rc2, 0.8.0, 0.8.1, 0.9.0, 0.9.1, 0.10.0, 0.10.1, 0.11.0, 0.12.0, 0.13.0, 0.13.1, 0.14.0, 0.14.1, 0.15.0, 0.15.1, 0.15.2, 0.16.0, 0.16.1, 0.16.2, 0.17.0, 0.17.1, 0.18.0, 0.18.1, 0.19.0rc1, 0.19.0, 0.19.1, 0.19.2, 0.20.0rc1, 0.20.0, 0.20.1, 0.20.2, 0.20.3, 0.21.0rc1, 0.21.0, 0.21.1, 0.22.0, 0.23.0rc2, 0.23.0, 0.23.1, 0.23.2, 0.23.3, 0.23.4)
No matching distribution found for pandas==1.1.0
You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [5]:
from fastscoredeploy import ipmagic

In [6]:
from autoencoder import load_model

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


#from models.recommender.autorecommender.models.autoencoder import load_model # need this to load the model

In [7]:
DATA_PATH = '/home/jovyan/work/POC/models/data/'
STOCKS_CSV = DATA_PATH + 'stocks.csv'
USERS_CSV = DATA_PATH + 'train_users.csv'
RATINGS_CSV = DATA_PATH + 'train_ratings.csv'

In [8]:
RATINGS = pd.read_csv(RATINGS_CSV)
RATINGS = RATINGS.set_index(['userid', 'stockid'])

STOCKS = pd.read_csv(STOCKS_CSV)
STOCKS = STOCKS.set_index('stockid').sort_index()

STOCKS_TICKER_LOOKUP = STOCKS.reset_index().set_index('ticker')

USERS = pd.read_csv(USERS_CSV).set_index('userid')

How to make a prediction with the model

In [9]:
def stockid_to_stockname(number):
    x = STOCKS.loc[number]
    return x['ticker']

In [40]:
%%schema ratings
{
    "type": "record",
    "name": "ratings",
    "fields":[
        {"type": "string", "name": "userid"},
        {"type": {"type": "map", "values": "double"},"name":"ratings"}
    ]
}

Schema loaded and bound to ratings variable


In [41]:
%%schema output
{
    "type": "array",
    "items": "string"
}

Schema loaded and bound to output variable


In [12]:
# convert input to appropriate form for the model

def df_from_fastscore(x):
    userid = x['userid']
    ratings_list = []
    for r in x['ratings']:
        key = list(r.keys())[0]
        val = list(r.values())[0]
        ratings_list.append({'userid': userid, 
                             'stockid': STOCKS_TICKER_LOOKUP.loc[key]['stockid'].iloc[0], 
                             'rating' : val})
    return pd.DataFrame(ratings_list).set_index(['userid', 'stockid'])

This is a dataframe with a multiindex (userid, stockid) with a column named "rating" --> this is what the inputs FastScore receives need to be translated into.

In [13]:
cd /home/jovyan/work/POC/models/notebooks/

/home/jovyan/work/POC/models/notebooks


In [14]:
trained_model = load_model('model.zip')

In [15]:
model_inputs = df_from_fastscore({'userid': 123,
                                 'ratings': [
                                     {'AABA': 4},
                                     {'AAL': 3},
                                     {'AAON': 5}
                                 ]})

In [16]:
predictions = trained_model.predict(model_inputs)

In [17]:
def predictions_to_recommendations(preds, userid):
    stockids = preds.transpose().sort_values(by=userid, ascending=False).index[0:10]
    outs = [stockid_to_stockname(x) for x in stockids]
    return outs
    

In [18]:
predictions_to_recommendations(predictions, 123)

['LACQU',
 'PME',
 'VCSH',
 'ELTK',
 'BRPAU',
 'VRNT',
 'CVLY',
 'UCBA',
 'XCRA',
 'CGO']

## FastScore Action function

In [49]:
%%model encoder
# fastscore.schema.0: ratings
# fastscore.schema.1: output
# fastscore.module-attached: autorecommender

from autorecommender.models.autoencoder import load_model

def begin():
    global trained_model, STOCKS, STOCKS_TICKER_LOOKUP
    trained_model = load_model('model.zip')
    
    DATA_PATH = '' # Update this if necessary
    STOCKS_CSV = DATA_PATH + 'stocks.csv'

    STOCKS = pd.read_csv(STOCKS_CSV)
    STOCKS = STOCKS.set_index('stockid').sort_index()

    STOCKS_TICKER_LOOKUP = STOCKS.reset_index().set_index('ticker')

def df_from_fastscore(x):
    userid = x['userid']
    ratings_list = []
    for r in x['ratings']:
        key = list(r.keys())[0]
        val = list(r.values())[0]
        ratings_list.append({'userid': userid, 
                             'stockid': STOCKS_TICKER_LOOKUP.loc[key]['stockid'].iloc[0], 
                             'rating' : val})
    return pd.DataFrame(ratings_list).set_index(['userid', 'stockid'])
    
def predictions_to_recommendations(preds, userid):
    stockids = preds.transpose().sort_values(by=userid, ascending=False).index[0:10]
    outs = [stockid_to_stockname(x) for x in stockids]
    return outs

def stockid_to_stockname(number):
    x = STOCKS.loc[number]
    return x['ticker']

def action(x):
    userid = x['userid']
    inputs = df_from_fastscore(x)
    yield predictions_to_recommendations(trained_model.predict(inputs), userid)

ModuleNotFoundError: No module named 'autorecommender'

In [43]:
from fastscoredeploy.suite import Connect

c = Connect('https://dashboard:8000')
mm = c.lookup('model-manage')
eng = c.lookup('engine')

In [44]:
encoder.update(model_manage=mm)

True

In [45]:
from fastscore.attachment import Attachment

att = Attachment('att', datafile='model.zip')
att.upload(encoder)

In [46]:
#sample input 
next(action({'userid': 123,
             'ratings': [
                 {'AABA': 4},
                 {'AAL': 3},
                 {'AAON': 5}
             ]}))

NameError: name 'action' is not defined

In [47]:
encoder.deploy(eng)

In [ ]:
#eng.score(inputs.to_dict(orient='records')[0:25])

In [48]:
eng.score({'userid': 123,
             'ratings': [
                 {'AABA': 4},
                 {'AAL': 3},
                 {'AAON': 5}
             ]})

NameError: name 'FastScoreError' is not defined

## Fastscore Begin Function

In [22]:
def begin():
    global trained_model, STOCKS, STOCKS_TICKER_LOOKUP
    trained_model = load_model('model.zip')
    
    DATA_PATH = '../data/' # Update this if necessary
    STOCKS_CSV = DATA_PATH + 'stocks.csv'

    STOCKS = pd.read_csv(STOCKS_CSV)
    STOCKS = STOCKS.set_index('stockid').sort_index()

    STOCKS_TICKER_LOOKUP = STOCKS.reset_index().set_index('ticker')

Other functions to include in the FastScore model execution script:

* `stockid_to_stockname`
* `predictions_to_recommendations`
* `df_from_fastscore`

Files that need to be in the attachment:

* `model.zip`
* `stocks.csv`